# Afonichev Demid SSE2019

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

In [2]:
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat,Survived
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,2,Yes
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,Yes
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No



## Start with basic EDA (Exploratory data analysis): 
- Compute average `Age` of passengers and number of passengers who survived and not survived grouped by `Sex` and `Passenger Class` (24 numbers);
- What can you say about survivors based on the resulting table (open question), e.g. what is the surviving ratio for females in First class compared to the Second and Third? *This answer is limited to 150 words.*
- What is the average number of males and females on all boats (rounded to the closest integer)? *Do not forget to filter out all `?` in `Life Boat` attribute.*

In [12]:
titanic.groupby(['Sex', 'Passenger Class', 'Survived']).agg(mean_age=('Age', 'mean'), size=('Age', 'size'))

mean_age  size
Sex    Passenger Class Survived                 
Female First           No        35.200000     5
                       Yes       37.109375   139
       Second          No        34.090909    12
                       Yes       26.711051    94
       Third           No        23.418750   110
                       Yes       20.814815   106
Male   First           No        43.658163   118
                       Yes       36.168240    61
       Second          No        33.092593   146
                       Yes       17.449274    25
       Third           No        26.679598   418
                       Yes       22.436441    75

- The number of women survivors exceeds the number of men survivors for each class.
- The probability of surviving is ranged by class as follows (largest to lowest): first, third, second for both sex.
- The probability of surviving being a woman is greater than 50% for first (96%) and second (87%) class
- The probability of surviving being a man is lower than 50% for all classes
- The lower the class, the lower is the probability of surviving (except men from third class against men from second class)
- The mean age of non-survivors exceeds the mean age of non-survivors for each sex and class except women from first class
- The mean age over classes is ranged as follows: first, second, third (except men survivors from third class)

In [13]:
on_boat = titanic[titanic['Life Boat'].isna() == False]

on_boat = on_boat.groupby(['Life Boat', 'Sex']).agg(size=('Age', 'size')).reset_index()
on_boat = on_boat.groupby(['Sex']).agg(mean=('size', lambda x: math.ceil(sum(x) / len(x)))).reset_index()
on_boat.head()

,Sex,mean
0,Female,14
1,Male,7


## Proceed with feature generation.
- Drop the column `Life Boat`.
- Generate new attribute `Family size` (sum up `No of Parents or Children on Board` and `No of Siblings or Spouses on Board`). What is the average family size? In which class did the biggest family travel?
Do not drop original attributes.
- It seems that `Passenger Fare` is total among all passengers with the same `Ticket Number`: create new attribute `Single passenger fare`. For every passenger you need to compute the number of passengers with the same `Ticket Number` and then use this number as a divisor for `Passenger Fare`. *Do not drop the original attribute.*
- Impute missing values: for numerical attributes use averaging over three groups: `Passenger Class`, `Sex`, `Embarkation Port`; for every numerical attribute create separate column that contain 1 for imputed value and 0 for originally presented. 
- Pre-process categorical attributes: For every categorical attribute create a separate column that contains 1 for imputed value and 0 for originally presented. One-hot encode categorical attributes with less than 20 unique values, drop other categorical attributes; drop original attributes. 
- Set the role of the `Survived` attribute to `label`.

## Finish by building a classification model using preprocessed data
- Compute classification accuracy on a train-test setup:
    - Create a Cross Validation block, fix the random_state parameter to 2020.
    - Use a decision tree with `maximal depth` = 7; uncheck `apply pruning` box; leave all other parameters by default.
    - Use accuracy as a performance metric
- Analyze the resulting confusion matrix, which error is larger: Type I or Type II? 
- Provide a short analysis of the results, based on your answers III.2-III.3. E.g. What are the splitting features of the first 3 levels of the best tree (up to 7 attributes)? Do these results coincide with your intuition? You may include some misclassified examples along with explanations why they were misclassified. *This answer is limited to 250 words.*